# Hyperparameter tuning
Tune the weights of the weighted functions used in SynBA transformation (`WordSwapMultimodal`).

To do so, use [HyperOpt](https://github.com/hyperopt/hyperopt) library.
TPE is a default algorithm for the Hyperopt. It uses Bayesian approach for optimization. At every step it is trying to build probabilistic model of the function and choose the most promising parameters for the next step. Generally this types of algorithms works like this:

1. Generate random initial point  $x^∗$ 
2. Calculate  $F(x^∗)$
3. Using the history of trials try to build the conditional probability model  $P(F|x)$ 
4. Choose  $x_i$  that according to the  $P(F|x)$  will most probably result in better  $F(x_i)$
5. Compute the real value of the  $F(x_i)$ 
6. Repeat steps 3-5 until one of the stop criteria is satisfied, for example $i > max_eval$

In [1]:
import random
import torch
import gc

import transformers
from transformers import logging
logging.set_verbosity_error() # disable transformers logging

from textattack.attack_recipes import SynBA2022

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper

from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    MaxModificationRate,
    RepeatModification,
    StopwordModification,
    InputColumnModification
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import BERT
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapMultimodal
from textattack import Attacker
from textattack import AttackArgs

from textattack.metrics.quality_metrics import (
    SBERTMetric,
    ContradictionMetric
)
from textattack.metrics.attack_metrics import (
    AttackSuccessRate
)

# Import HyperOpt Library
from hyperopt import tpe, hp, fmin

c:\Users\peppe\anaconda3\envs\3rdplace\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import the model
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-rotten-tomatoes")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-rotten-tomatoes")
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Import the dataset
dataset_name = "rotten_tomatoes"
dataset = HuggingFaceDataset(dataset_name, None, "test")

# whether to make the attack reproducible or set it random for each run
random_samples = False
random_seed = random.randint(0, 10000) if random_samples else 445

# set the number of samples to attack
num_examples=500

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (C:\Users\peppe\.cache\huggingface\datasets\rotten_tomatoes_movie_review\default\1.0.0\e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5)
100%|██████████| 3/3 [00:00<00:00, 500.04it/s]
textattack: Loading datasets dataset rotten_tomatoes, split test.


In [3]:
def build_synba(model_wrapper, lambda_mlm, lambda_thesaurus, lambda_we):
    """
    Build the attack, with the given hyperparameters
    """
    #
    # Candidate size K is set to 30 for all data-sets.
    transformation = WordSwapMultimodal(max_candidates=30, multimodal_weights=(lambda_mlm, lambda_thesaurus, lambda_we), min_confidence=0.1)
    #
    # Don't modify the same word twice or stopwords.
    #
    constraints = [RepeatModification(), StopwordModification()]
    input_column_modification = InputColumnModification(
        ["premise", "hypothesis"], {"premise"}
    )
    constraints.append(input_column_modification)
    constraints.append(MaxModificationRate(max_rate=0.3, min_threshold=4))
    constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
    constraints.append(WordEmbeddingDistance(min_cos_sim=0.6, include_unknown_words=True))
    constraints.append(BERT(model_name="stsb-mpnet-base-v2", threshold=0.7, metric="cosine"))
    #
    # Goal is untargeted classification
    #
    goal_function = UntargetedClassification(model_wrapper)
    #
    # Greedily swap words with "Word Importance Ranking".
    #
    search_method = GreedyWordSwapWIR(wir_method="gradient")

    return Attack(goal_function, constraints, transformation, search_method)

def perform_attack(model, dataset, num_examples, random_seed, lambda_mlm, lambda_thesaurus, lambda_we):
    """
    Set up the attack and perform it on the dataset.
    Returns the metrics that we want to optimize.
    """
    attack = build_synba(model, lambda_mlm, lambda_thesaurus, lambda_we)
    attack_args = AttackArgs(num_examples=num_examples, shuffle=False, random_seed=random_seed, disable_stdout=True, silent=True)
    attacker = Attacker(attack, dataset, attack_args)
    attack_results = attacker.attack_dataset()

    # compute metrics
    attack_success_stats = AttackSuccessRate().calculate(attack_results)
    sbert_stats = SBERTMetric().calculate(attack_results)["avg_attack_sentence_bert_similarity"]
    contradiction_stats = ContradictionMetric(by_sentence=True).calculate(attack_results)["attack_contradiction_rate"]

    # free memory after performing the attack
    gc.collect()
    torch.cuda.empty_cache()
    return attack_success_stats, sbert_stats, contradiction_stats

In [4]:
def optimize(args):
    """
    This is the function that hyperopt will maximize.
    """
    # make sure that the hyperparameters sum up to 1
    w1 = args["w1"]
    w2 = (1 - args["w1"]) * args["w2"]
    w3 = (1 - args["w1"]) * (1 - args["w2"])

    print(f"Now tring --> lambda_mlm: {round(w1, 4)}, lambda_thesaurus: {round(w2, 4)}, lambda_we: {round(w3, 4)}")
    attack_success_stats, sbert_similarity, contraddiction_rate = perform_attack(model_wrapper, dataset, num_examples, random_seed, w1, w2, w3)
    
    print(f"Succ/Fail/Skip: {attack_success_stats['successful_attacks']}/{attack_success_stats['failed_attacks']}/{attack_success_stats['skipped_attacks']}")
    print(f"--> SBERT Similarity: {sbert_similarity}, Contraddiction Rate: {contraddiction_rate}")
    
    # compute penality for failed attacks
    penalty = 0.2 * (attack_success_stats["failed_attacks"] / (attack_success_stats["failed_attacks"]+attack_success_stats["successful_attacks"]))

    # objective to maximize (minimize the negative)
    loss = -(sbert_similarity * (1-contraddiction_rate)) + penalty
    print(f"--> Loss: {loss}\n")
    
    return loss

In [5]:
# define the search space across the hyperparameters
space = hp.choice(
    "weights",
    [
        {
            "w1": hp.uniform("w1", 0, 1),
            "w2": hp.uniform("w2", 0, 1),
        },
    ],
)

best = fmin(
    optimize,
    space,
    algo=tpe.suggest,
    max_evals=100,
)

print("\nThe best combination of hyperparameters is:")
w1 = best["w1"]
w2 = (1 - best["w1"]) * best["w2"]
w3 = (1 - best["w1"]) * (1 - best["w2"])
print(f"lambda_mlm: {round(w1, 4)}, lambda_thesaurus: {round(w2, 4)}, lambda_we: {round(w3, 4)}")

Now tring --> lambda_mlm: 0.622, lambda_thesaurus: 0.0763, lambda_we: 0.3018
Succ/Fail/Skip: 323/83/94                              
--> SBERT Similarity: 0.899, Contraddiction Rate: 0.139
--> Loss: -0.7331523004926108                          

Now tring --> lambda_mlm: 0.2177, lambda_thesaurus: 0.1074, lambda_we: 0.6749          
Succ/Fail/Skip: 327/79/94                                                              
--> SBERT Similarity: 0.897, Contraddiction Rate: 0.147                                
--> Loss: -0.7262247438423645                                                          

Now tring --> lambda_mlm: 0.9883, lambda_thesaurus: 0.0028, lambda_we: 0.0088          
Succ/Fail/Skip: 16/390/94                                                              
--> SBERT Similarity: 0.966, Contraddiction Rate: 0.062                                
--> Loss: -0.7139897733990147                                                          

Now tring --> lambda_mlm: 0.9365, lambda_thesaur

In [10]:
best

{'w1': 0.8398808190105614, 'w2': 0.27544764840994906, 'weights': 0}